# 전처리

In [10]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2022-06-19

In [11]:
# 머신러닝 맛보기. 분류문제
import pandas as pd
data = pd.read_csv("breast-cancer-wisconsin.csv")

x = data[data.columns[1:10]]
y = data[["Class"]]

from sklearn.model_selection import train_test_split
# stratify = y 는 분류문제에서 필수적이다. 예측에서는 사용X.
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, stratify = y)

# print(y_train.mean())
# print(y_test.mean())

# MinMaxScaler를 이용해 정규화 하는 방법. 최소값 0 최대값 1로 정규화된다.
from sklearn.preprocessing import MinMaxScaler
scaler_minmax = MinMaxScaler()
scaler_minmax.fit(x_train)
x_scaled_minmax_train = scaler_minmax.transform(x_train)
x_scaled_minmax_test = scaler_minmax.transform(x_test)


# StandardScaler를 이용해 정규화 하는 방법. 평균 0 표준편차 1로 정규화된다.
# from sklearn.preprocessing import StandardScaler
# scaler_standard = StandardScaler()
# scaler_standard.fit(x_train)
# x_scaled_standard_train = scaler_standard.transform(x_train)
# x_scaled_standard_test = scaler_standard.transform(x_test)

# LogisticRegression 모듈을 이용해 학습.
# 로지스틱회귀는 대표적인 분류학습 모델이다. 여기서 쓰기 딱 좋음.
# (선형회귀는 분류학습이 아닌 예측학습이다 알지?)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_scaled_minmax_train, y_train)

pred_train = model.predict(x_scaled_minmax_train)
model.score(x_scaled_minmax_train, y_train)

pred_test = model.predict(x_scaled_minmax_test)
model.score(x_scaled_minmax_test, y_test)

# 혼동행렬을 사용하여 얼마나 맞추고 얼마나 틀렸는지 확인가능.
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
# print("훈련데이터 혼동행렬 : \n", confusion_train)
confusion_test = confusion_matrix(y_test, pred_test)
# print("훈련데이터 혼동행렬 : \n", confusion_test)

# 혼동행렬로 정확도 재현도 지지도 f1-score 등이 보고 싶다면 classification_report을 사용.
from sklearn.metrics import classification_report
report = classification_report(y_train, pred_train)
# print("classification_report : \n", report)

# roc_auc 점수가 보고싶다면 아래와 같은 코드.
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test, model.decision_function(x_scaled_minmax_test))
# print(roc_auc)


# 모든게 다 끝났으면 예측값과 확률을 원데이터와 병합하고 csv파일로 저장하여야 한다.
# !!주의!! y_train에 "y_pred"를 병합시킬때 pred_train은 그냥은 안붙는다. 대괄호 1개만 쓸것.
prob_train = model.predict_proba(x_scaled_minmax_train)
y_train["y_pred"] = pred_train
y_train[["y_prob0", "y_prob1"]] = prob_train
# print(y_train)

prob_test = model.predict_proba(x_scaled_minmax_test)
y_test["y_pred"] = pred_test
y_test[["y_prob0", "y_prob1"]] = prob_test
# print(y_test)

# 맨 처음 x와 y로 분리했던거 다시 붙이려면 pandas의 concat 메소드 사용.
total_test = pd.concat([x_test, y_test], axis = 1)
# print(total_test)

total_test.to_csv("classification_test.csv")

In [12]:
# 머신러닝 맛보기. 회귀문제
import pandas as pd
data = pd.read_csv("house_price.csv")

data.head()

x = data[["housing_age", "income", "bedrooms", "households", "rooms"]]
y = data[["house_value"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)

from sklearn.preprocessing import MinMaxScaler
scaler_minmax = MinMaxScaler()
scaler_minmax.fit(x_train)
x_scaled_minmax_train = scaler_minmax.transform(x_train)
x_scaled_minmax_test = scaler_minmax.transform(x_test)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_scaled_minmax_train, y_train)
pred_train = model.predict(x_scaled_minmax_train)
print(model.score(x_scaled_minmax_train, y_train))
pred_test = model.predict(x_scaled_minmax_test)
print(model.score(x_scaled_minmax_test, y_test))

y_train[["y_pred"]] = pred_train
y_test[["y_pred"]] = pred_test

total_test = pd.concat([x_test, y_test], axis = 1)
# print(total_test)
total_test.to_csv("regression_test.csv")

0.5706921210926263
0.5826083517811866


In [13]:
# 머신러닝 프로세스 - 1. 범주변수의 변환
import pandas as pd
data = pd.read_csv("vote.csv")

# 맨 처음에 할 일은 범주변수와 그렇지 않은 모든변수 2가지 종류로 나누는 것. (나중에 다시 붙일것이다.)
x1 = data[["gender", "region"]]
xy = data[["edu", "income", "age", "score_gov", "score_progress", "score_intention", "vote", "parties"]]

# 각각의 범주변수에 대해 의미를 부여하는 문자열로 변경한다.
x1["gender"] = x1["gender"].replace([1, 2], ["male", "female"])
x1["region"] = x1["region"].replace([1, 2, 3, 4, 5], ["Sudo", "Chungcheung", "Honam", "Youngnam", "Others"])

# 이 한줄로 언더바가 붙는 원 핫 인코딩이 완성된다.
x1_dum = pd.get_dummies(x1)
# print(x1_dum.head())

# 이제 붙여야지.
f_vote = pd.concat([x1_dum, xy], axis = 1)
# print(f_vote.head())
f_vote.to_csv("f_vote.csv", index = False)

In [14]:
# 머신러닝 프로세스 - 2. 데이터셋 분할과 모델검증
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv("Fvote.csv")

x = data[data.columns[:13]]
y = data[["vote"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()


# 데이터셋을 분할하는 방법은 3가지 정도 있다.
# 1. 랜덤 없는 교차검증 cross_val_score
# 2. 랜덤 있는 교차검증 K-Fold
# 3. 임의분할 교차검증 ShuffleSplit
# 모두 model_selection 패키지 안에 있다.

from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit
scores = cross_val_score(model, x_train, y_train, cv = 5)
# print(scores)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)
scores = cross_val_score(model, x_train, y_train, cv = kfold)
# print(scores)
shuffle_split = ShuffleSplit(n_splits = 5, test_size = 0.5, random_state = 42)
scores = cross_val_score(model, x_train, y_train, cv = shuffle_split)
# print(scores)

model.fit(x_train, y_train)
model.score(x_test, y_test)

0.6792452830188679

In [15]:
# 머신러닝 프로세스 - 3. 데이터 정규화
import pandas as pd
data = pd.read_csv("Fvote.csv")

x = data.loc[:, "gender_female":"score_intention"]
y = data[["vote"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()

scaler_minmax.fit(x_train)
x_scaled_minmax_train = scaler_minmax.transform(x_train)
x_scaled_minmax_test = scaler_minmax.transform(x_test)

scaler_standard.fit(x_train)
x_scaled_standard_train = scaler_standard.transform(x_train)
x_scaled_standard_test = scaler_standard.transform(x_test)


from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(x_scaled_minmax_train, y_train)
pred_train = model.predict(x_scaled_minmax_train)
pred_test = model.predict(x_scaled_minmax_test)
model.score(x_scaled_minmax_test, y_test)

model.fit(x_scaled_standard_train, y_train)
pred_train = model.predict(x_scaled_standard_train)
pred_test = model.predict(x_scaled_standard_test)
model.score(x_scaled_standard_test, y_test)

0.6792452830188679

# 2022-06-20

In [38]:
# 머신러닝 프로세스 - 4. 모델훈련과 세부튜닝
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv("Fvote.csv")
x = data[data.columns[:13]]
y = data[["vote"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

# 그리드 서치
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv = 5, return_train_score = True)
grid_search.fit(x_train, y_train)

print(f"Best Parameter : {grid_search.best_params_}")
print(f"Best Cross-validity Score : {grid_search.best_score_}")
print(f"Test set Score : {grid_search.score(x_test, y_test)}")
print(f"{pd.DataFrame(grid_search.cv_results_)}")

print()
print("----------------------------------------")
print()

# 랜덤 서치
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distrib = {"C": randint(low = 0.001, high = 100)}

random_search = RandomizedSearchCV(LogisticRegression(), param_distrib, cv = 5, return_train_score = True)
random_search.fit(x_train, y_train)
print(f"Best Parameter : {random_search.best_params_}")
print(f"Best Cross-validity Score : {random_search.best_score_}")
print(f"Test set Score : {random_search.score(x_test, y_test)}")
print(f"{pd.DataFrame(random_search.cv_results_)}")

Best Parameter : {'C': 10}
Best Cross-validity Score : 0.7274193548387096
Test set Score : 0.6792452830188679
   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.009386      0.000488         0.002936        0.000075   0.001   
1       0.010455      0.000300         0.002830        0.000045    0.01   
2       0.018721      0.004374         0.003090        0.000099     0.1   
3       0.024299      0.002630         0.003132        0.000044       1   
4       0.031231      0.003640         0.003270        0.000104      10   
5       0.034626      0.003736         0.003211        0.000077     100   

         params  split0_test_score  split1_test_score  split2_test_score  \
0  {'C': 0.001}            0.68750            0.71875            0.71875   
1   {'C': 0.01}            0.68750            0.71875            0.71875   
2    {'C': 0.1}            0.71875            0.71875            0.71875   
3      {'C': 1}            0.71875            0.68750       

In [58]:
# 머신러닝 프로세스 - 5. 모델평가
import pandas as pd
data = pd.read_csv("Fvote.csv")

x = data[data.columns[:13]]
y = data[["vote"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C = 10)
model.fit(x_train, y_train)

pred_train = model.predict(x_train)
print(f"Score of train : {model.score(x_train, y_train)}")
pred_test = model.predict(x_test)
print(f"Score of test : {model.score(x_test, y_test)}")

from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print(confusion_train)
confusion_test = confusion_matrix(y_test, pred_test)
print(confusion_test)

from sklearn import metrics
roc_auc = metrics.roc_auc_score(y_test, model.decision_function(x_test))
print(f"roc_auc : {roc_auc}")

Score of train : 0.740506329113924
Score of test : 0.6792452830188679
[[ 12  34]
 [  7 105]]
[[ 3 12]
 [ 5 33]]
roc_auc : 0.6350877192982456


In [75]:
# 머신러닝 프로세스 - 6. 다중분류
# 일반 이진분류랑 사실 다를거 없다.

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv("Fvote.csv")

x = data[data.columns[:13]]
y = data[["parties"]]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C = 0.1)
model.fit(x_train, y_train)

model.score(x_train, y_train)
pred_train = model.predict(x_train)
model.score(x_test, y_test)
pred_test = model.predict(x_test)

from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print(confusion_train)

from sklearn.model_selection import GridSearchCV
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv = 5, return_train_score = True)
grid_search.fit(x_train, y_train)
print(f"가장 높은 그리드 서치 점수 : {grid_search.best_score_}")
print(f"가장 좋은 파라미터 : {grid_search.best_params_}")
print(f"테스트 점수 : {grid_search.score(x_test, y_test)}")

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distrib = {"C": randint(low = 0.001, high = 100)}
random_search = RandomizedSearchCV(LogisticRegression(), param_distrib, cv = 5, return_train_score = True)
random_search.fit(x_train, y_train)
print(f"가장 높은 랜덤 서치 점수 : {random_search.best_score_}")
print(f"가장 좋은 파라미터 : {random_search.best_params_}")
print(f"테스트 점수 : {random_search.score(x_test, y_test)}")

[[20  2  1 14]
 [ 1 25  2 12]
 [ 6  2  4  7]
 [ 6  8  1 47]]
가장 높은 그리드 서치 점수 : 0.55625
가장 좋은 파라미터 : {'C': 0.1}
테스트 점수 : 0.5471698113207547
가장 높은 랜덤 서치 점수 : 0.5185483870967742
가장 좋은 파라미터 : {'C': 84}
테스트 점수 : 0.5094339622641509
